In [1]:
import numpy as np
import pandas as pd
import scipy as scp
import itertools as it

In [4]:
class sys_resolve_atoms:
    def __init__(self, n):
        """
        Inicializa o sistema com n massas.

        Args:
            n (int): O número de massas (e o tamanho da matriz).
        """
        if n < 2:
            raise ValueError("O número de massas (n) deve ser pelo menos 2.")
        self.n = n
        self.matrix = np.zeros((n, n), dtype=float)
        self._autovalores = None
        self._autovetores = None

    def create_matrix(self, m: list, k: list):
        """
        Gera a matriz dinâmica D com base nas massas (m) e nas constantes de mola (k).
        """

        if len(m) != self.n:
            raise ValueError(f"A lista de massas 'm' deve ter {self.n} elementos, mas tem {len(m)}.")
        if len(k) != self.n:
            raise ValueError(f"A lista de constantes 'k' deve ter {self.n} elementos, mas tem {len(k)}.")

        
        self.matrix = np.zeros((self.n, self.n), dtype=float)

        for i in range(self.n):
            for j in range(self.n):
                if i == j:
                    if i == 0:
                        self.matrix[i, j] = (k[0] + k[self.n - 1])/ m[i]
                    elif i == self.n - 1:
                        self.matrix[i, j] = (k[i - 1] + k[self.n - 1]) / m[i]
                    else:
                        self.matrix[i, j] = (k[i - 1] + k[i]) / m[i]
                elif abs(i - j) == 1:
                    k_index = min(i, j)
                    self.matrix[i, j] = -k[k_index] / m[i]
                elif (i == self.n-1 and j == 0) or (i == 0 and j == self.n-1):
                    self.matrix[i, j] = -k[self.n - 1] / m[i]
        
        return self.matrix

    def _calculate_system_values(self):
        """
        Método interno para calcular autovalores e autovetores
        """
        autovalores, autovetores = np.linalg.eig(self.matrix)
        
        sorted_indices = np.argsort(autovalores)
        self._autovalores = autovalores[sorted_indices]
        self._autovetores = autovetores[:, sorted_indices]

    def get_autovalores(self):
        """
        Calcula e retorna os autovalores (λ = ω²) da matriz dinâmica D.
        """
        self._calculate_system_values()
        return self._autovalores

    def get_autovetores(self):
        """
        Calcula e retorna os autovetores (modos normais de vibração) da matriz D.

        Returns:
            np.ndarray: Uma matriz onde cada coluna é um autovetor correspondente
                        a um autovalor (na mesma ordem retornada por get_autovalores).
        """
        self._calculate_system_values()
        return self._autovetores


In [ ]:
df = pd.DataFrame()

In [30]:

n = 2

masses = [1.0, 1.0]
spring_constants = [1.0, 1.0]

# 1. Criar a instância do sistema
system = sys_resolve_atoms(n)

results = {}

for i in range(1, 11):
    masses[-1] = i

    results[f'm = {i}'] = {}

    print(f"\n*----------------------------------*\nM2 = {i}")


    # 2. Gerar a matriz dinâmica D
    results[f'm = {i}']['matriz'] = system.create_matrix(m=masses, k=spring_constants)
    print("--- Matriz Dinâmica D ---")
    print(results[f'm = {i}']['matriz'])

    # 3. Calcular e imprimir os autovalores (ω²)
    # Os autovalores correspondem ao quadrado das frequências angulares de oscilação
    results[f'm = {i}']['autovalores'] = system.get_autovalores()
    print("\n--- Autovalores (λ = ω²) ---")
    print(results[f'm = {i}']['autovalores'])

    # 4. Calcular e imprimir os autovetores (Modos Normais)
    # Cada coluna da matriz de autovetores representa um modo normal de vibração
    results[f'm = {i}']['autovetores'] = system.get_autovetores()
    print("\n--- Autovetores (Modos Normais de Vibração) ---")
    print(results[f'm = {i}']['autovetores'])

    # 5. Calcular as frequências angulares (ω)
    # ω = sqrt(λ), mas apenas para autovalores positivos
    positive_values = [ j for j in results[f'm = {i}']['autovalores'].reshape(-1) if j>= 0 ]
    results[f'm = {i}']['frequencias_angulares'] = np.sqrt(positive_values)
    print("\n--- Frequências Angulares de Oscilação (ω) ---")
    print(results[f'm = {i}']['frequencias_angulares'])


*----------------------------------*
M2 = 1
--- Matriz Dinâmica D ---
[[ 2. -1.]
 [-1.  2.]]

--- Autovalores (λ = ω²) ---
[1. 3.]

--- Autovetores (Modos Normais de Vibração) ---
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]

--- Frequências Angulares de Oscilação (ω) ---
[1.         1.73205081]

*----------------------------------*
M2 = 2
--- Matriz Dinâmica D ---
[[ 2.  -1. ]
 [-0.5  1. ]]

--- Autovalores (λ = ω²) ---
[0.6339746 2.3660254]

--- Autovetores (Modos Normais de Vibração) ---
[[ 0.59069049  0.9390708 ]
 [ 0.80689822 -0.34372377]]

--- Frequências Angulares de Oscilação (ω) ---
[0.79622522 1.538189  ]

*----------------------------------*
M2 = 3
--- Matriz Dinâmica D ---
[[ 2.         -1.        ]
 [-0.33333333  0.66666667]]

--- Autovalores (λ = ω²) ---
[0.45141623 2.21525044]

--- Autovetores (Modos Normais de Vibração) ---
[[ 0.54247681  0.97760877]
 [ 0.84007078 -0.21043072]]

--- Frequências Angulares de Oscilação (ω) ---
[0.67187516 1.48837174]

*--------

In [33]:
n = 3

masses = [1.0, 1.0, 1.0]
spring_constants = [1.0, 1.0, 1.0]

# 1. Criar a instância do sistema
system = sys_resolve_atoms(n)

results2 = {}

for i in ["Todas Iguais", "Uma Diferente", "Todas Diferentes"]:

    if i == "Todas Iguais":
        masses = [1.0, 1.0, 1.0]
    elif i == "Uma Diferente":
        masses = [1.0, 2.0, 1.0]
    elif i == "Todas Diferentes":
        masses = [1.0, 2.0, 3.0]
        

    results2[f'{i}'] = {}

    print(f"\n*----------------------------------*\n{i}")


    # 2. Gerar a matriz dinâmica D
    results2[f'{i}']['matriz'] = system.create_matrix(m=masses, k=spring_constants)
    print("--- Matriz Dinâmica D ---")
    print(results2[f'{i}']['matriz'])

    # 3. Calcular e imprimir os autovalores (ω²)
    # Os autovalores correspondem ao quadrado das frequências angulares de oscilação
    results2[f'{i}']['autovalores'] = system.get_autovalores()
    print("\n--- Autovalores (λ = ω²) ---")
    print(results2[f'{i}']['autovalores'])

    # 4. Calcular e imprimir os autovetores (Modos Normais)
    # Cada coluna da matriz de autovetores representa um modo normal de vibração
    results2[f'{i}']['autovetores'] = system.get_autovetores()
    print("\n--- Autovetores (Modos Normais de Vibração) ---")
    print(results2[f'{i}']['autovetores'])

    # 5. Calcular as frequências angulares (ω)
    # ω = sqrt(λ), mas apenas para autovalores positivos
    positive_values = [ j for j in results2[f'{i}']['autovalores'].reshape(-1) if j>= 0 ]
    results2[f'{i}']['frequencias_angulares'] = np.sqrt(positive_values)
    print("\n--- Frequências Angulares de Oscilação (ω) ---")
    print(results2[f'{i}']['frequencias_angulares'])


*----------------------------------*
Todas Iguais
--- Matriz Dinâmica D ---
[[ 2. -1. -1.]
 [-1.  2. -1.]
 [-1. -1.  2.]]

--- Autovalores (λ = ω²) ---
[-4.23796137e-16  3.00000000e+00  3.00000000e+00]

--- Autovetores (Modos Normais de Vibração) ---
[[-0.57735027  0.81649658  0.25359481]
 [-0.57735027 -0.40824829 -0.79893374]
 [-0.57735027 -0.40824829  0.54533893]]

--- Frequências Angulares de Oscilação (ω) ---
[1.73205081 1.73205081]

*----------------------------------*
Uma Diferente
--- Matriz Dinâmica D ---
[[ 2.  -1.  -1. ]
 [-0.5  1.  -0.5]
 [-1.  -1.   2. ]]

--- Autovalores (λ = ω²) ---
[0. 2. 3.]

--- Autovetores (Modos Normais de Vibração) ---
[[ 5.77350269e-01  5.77350269e-01  7.07106781e-01]
 [ 5.77350269e-01 -5.77350269e-01  3.83804137e-17]
 [ 5.77350269e-01  5.77350269e-01 -7.07106781e-01]]

--- Frequências Angulares de Oscilação (ω) ---
[0.         1.41421356 1.73205081]

*----------------------------------*
Todas Diferentes
--- Matriz Dinâmica D ---
[[ 2.         -1.

In [34]:
n = 3

masses = [1.0, 1.0, 1.0]
spring_constants = [1.0, 1.0, 1.0]

# 1. Criar a instância do sistema
system = sys_resolve_atoms(n)

results3 = {}

for i in ["Todas Iguais", "Uma Diferente", "Todas Diferentes"]:

    if i == "Todas Iguais":
        spring_constants = [1.0, 1.0, 1.0]
    elif i == "Uma Diferente":
        spring_constants = [1.0, 2.0, 1.0]
    elif i == "Todas Diferentes":
        spring_constants = [1.0, 2.0, 3.0]
        

    results3[f'{i}'] = {}

    print(f"\n*----------------------------------*\n{i}")


    # 2. Gerar a matriz dinâmica D
    results3[f'{i}']['matriz'] = system.create_matrix(m=masses, k=spring_constants)
    print("--- Matriz Dinâmica D ---")
    print(results3[f'{i}']['matriz'])

    # 3. Calcular e imprimir os autovalores (ω²)
    # Os autovalores correspondem ao quadrado das frequências angulares de oscilação
    results3[f'{i}']['autovalores'] = system.get_autovalores()
    print("\n--- Autovalores (λ = ω²) ---")
    print(results3[f'{i}']['autovalores'])

    # 4. Calcular e imprimir os autovetores (Modos Normais)
    # Cada coluna da matriz de autovetores representa um modo normal de vibração
    results3[f'{i}']['autovetores'] = system.get_autovetores()
    print("\n--- Autovetores (Modos Normais de Vibração) ---")
    print(results3[f'{i}']['autovetores'])

    # 5. Calcular as frequências angulares (ω)
    # ω = sqrt(λ), mas apenas para autovalores positivos
    positive_values = [ j for j in results3[f'{i}']['autovalores'].reshape(-1) if j>= 0 ]
    results3[f'{i}']['frequencias_angulares'] = np.sqrt(positive_values)
    print("\n--- Frequências Angulares de Oscilação (ω) ---")
    print(results3[f'{i}']['frequencias_angulares'])


*----------------------------------*
Todas Iguais
--- Matriz Dinâmica D ---
[[ 2. -1. -1.]
 [-1.  2. -1.]
 [-1. -1.  2.]]

--- Autovalores (λ = ω²) ---
[-4.23796137e-16  3.00000000e+00  3.00000000e+00]

--- Autovetores (Modos Normais de Vibração) ---
[[-0.57735027  0.81649658  0.25359481]
 [-0.57735027 -0.40824829 -0.79893374]
 [-0.57735027 -0.40824829  0.54533893]]

--- Frequências Angulares de Oscilação (ω) ---
[1.73205081 1.73205081]

*----------------------------------*
Uma Diferente
--- Matriz Dinâmica D ---
[[ 2. -1. -1.]
 [-1.  3. -2.]
 [-1. -2.  3.]]

--- Autovalores (λ = ω²) ---
[-4.88831496e-16  3.00000000e+00  5.00000000e+00]

--- Autovetores (Modos Normais de Vibração) ---
[[-5.77350269e-01  8.16496581e-01 -1.69434021e-16]
 [-5.77350269e-01 -4.08248290e-01 -7.07106781e-01]
 [-5.77350269e-01 -4.08248290e-01  7.07106781e-01]]

--- Frequências Angulares de Oscilação (ω) ---
[1.73205081 2.23606798]

*----------------------------------*
Todas Diferentes
--- Matriz Dinâmica D --

In [25]:
[ j for j in results[f'm = {i}']['autovetores'].reshape(-1) if j>= 0 ]
    

[0.7071067811865475, 0.7071067811865475, 0.7071067811865475]

In [20]:
i

array([ 0.88012902, -0.05386091])

In [19]:
results[f'm = {i}']

KeyError: 'm = [ 0.88012902 -0.05386091]'

In [18]:
results[f'm = {i}']['autovetores'].reshape(-1)

KeyError: 'm = [ 0.88012902 -0.05386091]'